In [1]:
import findspark
findspark.init(spark_home = "/home/thanhphat/BigData/spark-3.5.0-bin-hadoop3")

In [2]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as f
import pyspark.sql.types as t

In [3]:
# Create SparkSession
spark = SparkSession.builder.master("local[4]") \
    .appName("Test_log_delta") \
    .config("spark.sql.warehouse.dir", f"hdfs://localhost:9000/lakehouse/warehouse/LH_Global_Electronics_Retailer") \
    .config("spark.sql.catalogImplementation", "hive").enableHiveSupport() \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.parquet.vorder.enabled", "true") \
    .config("spark.sql.shuffle.partitions", 100) \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

spark.sparkContext.getConf().getAll()

24/06/29 13:57:23 WARN Utils: Your hostname, thanhphat-inspiron-5406-2n1 resolves to a loopback address: 127.0.1.1; using 192.168.1.8 instead (on interface wlp0s20f3)
24/06/29 13:57:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/29 13:57:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.executor.memory', '2g'),
 ('spark.app.submitTime', '1719644246138'),
 ('spark.master', 'local[4]'),
 ('spark.app.name', 'Test_log_delta'),
 ('spark.driver.host', '192.168.1.8'),
 ('spark.app.id', 'local-1719644255153'),
 ('spark.driver.port', '37819'),
 ('spark.sql.warehouse.dir',
  'hdfs://localhost:9000/lakehouse/warehouse/LH_Global_Electronics_Retailer'),
 ('spark.app.startTime', '1719644246837'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.cores', '2'),
 ('spark.sql.shuffle.partitions', '100'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAM

In [6]:
df = spark.read.format("parquet").load("hdfs://localhost:9000/lakehouse/LH_Global_Electronics_Retailer/Files/log/2024-06-29/batch_8")

df.show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------+-----------------+----------------+-------+-------------+----------+-----+----------------+
|BatchId|            TaskName|    SourceConnection|      SourceDatabase|         SourceTable|         TargetTable|          StartTime|            EndTime|SourceRowsRead|NumTargetInserted|NumTargetUpdated| Status|ColumnMissing|ColumnNull|Error|           Phase|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------+-----------------+----------------+-------+-------------+----------+-----+----------------+
|      8|Load Exchange_Rat...|LH_Global_Electro...|LH_Global_Electro...|parquet_exchange_...|silver_exchange_r...|2024-06-29 12:15:19|                   |         11215|            11215|               0|Success|     

In [5]:
spark.sql(f"USE lh_global_electronics_retailer")

spark.sql("DESCRIBE HISTORY silver_customer").show(truncate = False)

24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.metastore.wm.default.pool.size does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.llap.task.scheduler.preempt.independent does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.llap.output.format.arrow does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.tez.llap.min.reducer.per.executor does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.arrow.root.allocator.limit does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.vectorized.use.checked.expressions does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.tez.dynamic.semijoin.reduction.for.mapjoin does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.vectorized.complex.types.enabled does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.server2.wm.worker.threads does not exist
24/06/29 13:58:02 WARN HiveConf: HiveConf of name hive.repl.partitions.dump.

+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                 |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------------+------------+-----------------------------------+
|0      |2024-06-29 12:13:59.648|NULL  |NULL    |WRITE    |{mode -> Overwrite, partitionBy -> []}|NULL|NULL    |NULL     |NULL       |Serializable  |false        |